In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
columns_to_drop = ['PoolQC','Fence','MiscFeature','FireplaceQu','Alley','Id']
train = train.drop(columns_to_drop, axis = 1)
test = test.drop(columns_to_drop, axis = 1)

In [4]:
test['LotFrontage'] = test['LotFrontage'].fillna(test['LotFrontage'].mean())
test['MasVnrArea'] = test['MasVnrArea'].fillna(test['MasVnrArea'].mean())
train['LotFrontage'] = train['LotFrontage'].fillna(train['LotFrontage'].mean())
train['MasVnrArea'] = train['MasVnrArea'].fillna(train['MasVnrArea'].mean())

In [5]:
for col in list(test):
    test[col] = test[col].fillna(test[col].mode()[0])
for col in list(train):
    train[col] = train[col].fillna(train[col].mode()[0])

In [6]:
list_of_cat = list(train.select_dtypes(exclude=[np.number]))
len(list_of_cat)

38

In [7]:
train.shape

(1460, 75)

In [8]:
concat = pd.concat([train,test],axis = 0)
concat.shape
concat = concat.reset_index()
#concat = concat.loc[~concat.index.duplicated(keep='first')]

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
cat_df = concat[list_of_cat]
cat_encoder = OneHotEncoder()
cat_sparse = cat_encoder.fit_transform(cat_df)
cat_tran_df = pd.DataFrame(cat_sparse.toarray())
cat_tran_df.shape
cat_tran_df = cat_tran_df.reset_index()
#cat_tra_df = cat_tran_df.loc[~cat_tran_df.index.duplicated(keep='first')]

In [11]:
final_df = pd.concat([concat, cat_tran_df], axis = 1)
final_df.shape

(2919, 311)

In [12]:
list_of_cat2 = list(final_df.select_dtypes(exclude=[np.number]))
final_df = final_df.drop(list_of_cat2, axis = 1)
final_df = final_df.loc[:,~final_df.columns.duplicated()]

In [13]:
final_df.head()
final_df.columns = final_df.columns.astype(str)

In [14]:
final_df = final_df.drop('index', axis = 1)

In [15]:
#from sklearn.preprocessing import StandardScaler
#std_scale = StandardScaler()
#final_df_scaled = std_scale.fit_transform(final_df)
#final_df_scaled = pd.DataFrame(final_df_scaled)
#final_df_scaled.head()

In [16]:
final_train = final_df.loc[:1459,:]
final_test = final_df.loc[1460:,:]

In [17]:
final_train['SalePrice']

0       208500.0
1       181500.0
2       223500.0
3       140000.0
4       250000.0
          ...   
1455    175000.0
1456    210000.0
1457    266500.0
1458    142125.0
1459    147500.0
Name: SalePrice, Length: 1460, dtype: float64

In [18]:
final_test = final_test.drop('SalePrice',axis = 1)

In [19]:
#from sklearn.preprocessing import StandardScaler
#std_scale = StandardScaler()
#final_df_scaled = std_scale.fit_transform(final_df)
#final_df_scaled = pd.DataFrame(final_df_scaled)
#final_df_scaled.head()

In [20]:
X_train = final_train.drop('SalePrice',axis = 1)
y_train = final_train['SalePrice']

In [21]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [22]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train,y_train)

DecisionTreeClassifier()

In [23]:
import xgboost
c = xgboost.XGBRegressor()
c.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
pred = c.predict(final_test)

In [25]:
df = pd.DataFrame(pred)

In [26]:
df.to_csv('sub3.csv')